In [93]:
graphString = """

id  =simpleNode

id=node1
label=this is a label with an = sign 
title=multiline
title
thingy
URL = http://google.org

nodes=
color=red
shape=hexagon

edges=
color=RED

layout=
enabled=true

edges=
border=dotted
"""
lines = [line.strip() for line in graphString.split('\n\n') if line] #remove white matter
kvs = [line.split('=',1) for line in lines] #list of [k,v] 
stripped =  [[k.strip(), v.strip().split('\n')] for k,v in kvs] 
nodeData = [v for k,v in stripped if k=='id']
optData = [(k,v) for k,v in stripped if k!='id']
nodeData

[['simpleNode'],
 ['node1',
  'label=this is a label with an = sign ',
  'title=multiline',
  'title',
  'thingy',
  'URL = http://google.org']]

In [94]:
nodes=[]
for nodeDatum in nodeData:
    node=dict(id = nodeDatum[0])
    if len(nodeDatum)>1:
        remainder = nodeDatum[1:]
        for i,line in enumerate(remainder):
            if '=' in line:
                prop, value = line.split('=',1) 
                node[prop.strip()] = value
            else: #append the line to the previous value, as with multi-line labels 
                node[prop]+='\n' + line
            
    nodes.append(node)
nodes

[{'id': 'simpleNode'},
 {'id': 'node1',
  'label': 'this is a label with an = sign ',
  'title': 'multiline\ntitle\nthingy',
  'URL': ' http://google.org'}]

In [95]:
optData

[('nodes', ['color=red', 'shape=hexagon']),
 ('edges', ['color=RED']),
 ('layout', ['enabled=true']),
 ('edges', ['border=dotted'])]

In [90]:
options=dict(layout={}, nodes={}, edges={}, physics={})
for optLine in optData:
    k = optLine[0] #nodes, edges, etc.
    for v in optLine[1]:  
        #print(v)
        prop, value = v.split('=',1)                #color , red
        prop, value = prop.strip(), value.strip()   #color,red
        if value == 'true': value=True
        if value == 'false': value=False
        options[k][prop] = value                    #options['nodes']['color'] = red

for key in list(options.keys()): #remove e.g., options['physics']={} 
    if not options[key]:
        del options[key]
options        


{'layout': {'enabled': True},
 'nodes': {'color': 'red', 'shape': 'hexagon'},
 'edges': {'color': 'RED', 'border': 'dotted'}}

In [86]:
del options['physics']
options

{'layout': {'enabled': True},
 'nodes': {'color': 'red', 'shape': 'hexagon'},
 'edges': {'color': 'RED', 'border': 'dotted'}}

In [77]:
bool('false')

True

In [13]:
'a b c'.firstword()

AttributeError: 'str' object has no attribute 'firstword'